# Projet télédétection
Audrey Zimmer

## 1. Import des librairies

In [21]:
# Librairies python
import sys
sys.path.append('..')
import os
from sklearn.model_selection import train_test_split
import numpy as np
import plotly.express as px
from sklearn import tree
from sklearn.metrics import confusion_matrix, classification_report, \
    accuracy_score
import geopandas as gpd
from osgeo import ogr
from osgeo import gdal
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import RepeatedStratifiedKFold, KFold


# Librairies personnelles
from libsigma import classification as cla
from libsigma import read_and_write as rw
from libsigma import plots


## Création des dossiers

In [14]:
# Chemin de base (absolu)
my_folder = "/home/onyxia/work/Projet_teledetection"

# Chemins des dossiers à créer (absolus)
results_path = os.path.join(my_folder, "../results")
figure_path = os.path.join(my_folder, "../results/figure")
img_path = os.path.join(my_folder, "img")

# Création des dossiers
os.makedirs(results_path, exist_ok=True)
os.makedirs(figure_path, exist_ok=True)
os.makedirs(img_path, exist_ok=True)

print("Dossiers results, figure et img créés avec succès !")


Dossiers results, figure et img créés avec succès !


## Formatage des données

In [ ]:
b03 = '/home/onyxia/work/data/projet_eval/bretagne_23-24_B03.tif'
b03_data_set = gdal.Open(b03, gdal.GA_Update)
type(data_set)

b05 = '/home/onyxia/work/data/projet_eval/bretagne_23-24_B05.tif'
b05_data_set = gdal.Open(b05, gdal.GA_Update)
type(data_set)

osgeo.gdal.Dataset

In [16]:
print('Nombre de colonnes b03 :', data_set.RasterXSize)
print('Nombre de lignes b03 :', data_set.RasterYSize)
print('Nombre de bandes b03 :', data_set.RasterCount)

print('Nombre de colonnes b05 :', data_set.RasterXSize)
print('Nombre de lignes b05 :', data_set.RasterYSize)
print('Nombre de bandes b05 :', data_set.RasterCount)

Nombre de colonnes b03 : 1533
Nombre de lignes b03 : 612
Nombre de bandes b03 : 5
Nombre de colonnes b05 : 1533
Nombre de lignes b05 : 612
Nombre de bandes b05 : 5


Les bandes 03 et 05 qui sont utilisées pour la suite de l'étude contiennent le même nombre de colonnes (1533), le même nombre de lignes (612) et le même nombre de bandes (5).

In [5]:
# Liste des bandes
bandes = ['B02', 'B03', 'B04', 'B05', 'B06', 'B07', 'B08', 'B8A', 'B11', 'B12']

# Dictionnaire pour stocker les datasets
datasets = {}

# Vérification des dimensions
for bande in bandes:
    chemin = f'/home/onyxia/work/data/projet_eval/bretagne_23-24_{bande}.tif'
    datasets[bande] = gdal.Open(chemin, gdal.GA_ReadOnly)
    print(f"Bande {bande}:")
    print(f"  - Colonnes: {datasets[bande].RasterXSize}")
    print(f"  - Lignes: {datasets[bande].RasterYSize}")
    print(f"  - Nombre de bandes: {datasets[bande].RasterCount}\n")


Bande B02:
  - Colonnes: 1533
  - Lignes: 612
  - Nombre de bandes: 5

Bande B03:
  - Colonnes: 1533
  - Lignes: 612
  - Nombre de bandes: 5

Bande B04:
  - Colonnes: 1533
  - Lignes: 612
  - Nombre de bandes: 5

Bande B05:
  - Colonnes: 1533
  - Lignes: 612
  - Nombre de bandes: 5

Bande B06:
  - Colonnes: 1533
  - Lignes: 612
  - Nombre de bandes: 5

Bande B07:
  - Colonnes: 1533
  - Lignes: 612
  - Nombre de bandes: 5

Bande B08:
  - Colonnes: 1533
  - Lignes: 612
  - Nombre de bandes: 5

Bande B8A:
  - Colonnes: 1533
  - Lignes: 612
  - Nombre de bandes: 5

Bande B11:
  - Colonnes: 1533
  - Lignes: 612
  - Nombre de bandes: 5

Bande B12:
  - Colonnes: 1533
  - Lignes: 612
  - Nombre de bandes: 5



Toutes les bandes obtenues ont la même dimension : 
- 1533 colonnes
- 612 lignes
- 5 bandes

In [ ]:
# Chargement des datasets
for bande in bandes:
    chemin = f'/home/onyxia/work/data/projet_eval/bretagne_23-24_{bande}.tif'
    datasets[bande] = gdal.Open(chemin, gdal.GA_ReadOnly)

# Nombre de dates (5)
nb_dates = datasets['B02'].RasterCount

# Dimensions spatiales
cols = datasets['B02'].RasterXSize
rows = datasets['B02'].RasterYSize

# Pour chaque date
for date in range(1, nb_dates + 1):
    # Tableau 3D pour stocker toutes les bandes pour cette date
    cube_date = np.zeros((rows, cols, len(bandes)), dtype=np.float32)

    # Pour chaque bande
    for i, bande in enumerate(bandes):
        # Lecture de la bande pour cette date
        data = datasets[bande].GetRasterBand(date).ReadAsArray()
        # Stockage dans le cube
        cube_date[:, :, i] = data

    # Création du fichier de sortie
    driver = gdal.GetDriverByName('GTiff')
    chemin_sortie = f'/home/onyxia/work/results/bretagne_date_{date}.tif'
    out_dataset = driver.Create(chemin_sortie, cols, rows, len(bandes), gdal.GDT_Float32)

    # Copie des informations géoréférencées
    out_dataset.SetGeoTransform(datasets['B02'].GetGeoTransform())
    out_dataset.SetProjection(datasets['B02'].GetProjection())

    # Écriture des bandes
    for i in range(len(bandes)):
        out_band = out_dataset.GetRasterBand(i + 1)
        out_band.WriteArray(cube_date[:, :, i])
        out_band.FlushCache()

    print(f"Image pour la date {date} créée : {chemin_sortie}")

Image pour la date 1 créée : /home/onyxia/work/data/projet_eval/bretagne_date_1.tif
Image pour la date 2 créée : /home/onyxia/work/data/projet_eval/bretagne_date_2.tif
Image pour la date 3 créée : /home/onyxia/work/data/projet_eval/bretagne_date_3.tif
Image pour la date 4 créée : /home/onyxia/work/data/projet_eval/bretagne_date_4.tif
Image pour la date 5 créée : /home/onyxia/work/data/projet_eval/bretagne_date_5.tif


In [18]:
# Liste des bandes
bandes = ['B02', 'B03', 'B04', 'B05', 'B06', 'B07', 'B08', 'B8A', 'B11', 'B12']

# Liste des dates 
dates = ['2024-07-19', '2024-06-07', '2024-04-18', '2024-01-19', '2023-10-08']

# Dictionnaire pour stocker les datasets
datasets = {}

# Chargement des datasets
for bande in bandes:
    chemin = f'/home/onyxia/work/data/projet_eval/bretagne_23-24_{bande}.tif'
    datasets[bande] = gdal.Open(chemin, gdal.GA_ReadOnly)

# Dimensions spatiales
cols = datasets['B02'].RasterXSize
rows = datasets['B02'].RasterYSize

# Pour chaque date (indexé de 1 à 5)
for date_idx in range(1, len(dates) + 1):
    date = dates[date_idx - 1]  # Récupère la date correspondante
    # Tableau 3D pour stocker toutes les bandes pour cette date
    cube_date = np.zeros((rows, cols, len(bandes)), dtype=np.float32)

    # Pour chaque bande
    for i, bande in enumerate(bandes):
        # Lecture de la bande pour cette date
        data = datasets[bande].GetRasterBand(date_idx).ReadAsArray()
        cube_date[:, :, i] = data

    # Création du fichier de sortie (avec la date dans le nom)
    driver = gdal.GetDriverByName('GTiff')
    chemin_sortie = f'/home/onyxia/work/results/bretagne_{date}.tif'
    out_dataset = driver.Create(chemin_sortie, cols, rows, len(bandes), gdal.GDT_Float32)

    # Copie des informations géoréférencées
    out_dataset.SetGeoTransform(datasets['B02'].GetGeoTransform())
    out_dataset.SetProjection(datasets['B02'].GetProjection())

    # Écriture des bandes
    for i in range(len(bandes)):
        out_band = out_dataset.GetRasterBand(i + 1)
        out_band.WriteArray(cube_date[:, :, i])
        out_band.FlushCache()

    print(f"Image pour la date {date} créée : {chemin_sortie}")

Image pour la date 2024-07-19 créée : /home/onyxia/work/results/bretagne_2024-07-19.tif
Image pour la date 2024-06-07 créée : /home/onyxia/work/results/bretagne_2024-06-07.tif
Image pour la date 2024-04-18 créée : /home/onyxia/work/results/bretagne_2024-04-18.tif
Image pour la date 2024-01-19 créée : /home/onyxia/work/results/bretagne_2024-01-19.tif
Image pour la date 2023-10-08 créée : /home/onyxia/work/results/bretagne_2023-10-08.tif


## Analyse des échantillons
### Nombre d'échantillons

In [25]:
import geopandas as gpd
import plotly.express as px
import os
import pandas as pd

# Chemin vers le fichier shapefile
shp_path = '/home/onyxia/work/data/projet_eval/PI_strates_bretagne_32630.shp'

# Charger le fichier
gdf = gpd.read_file(shp_path)

# Compter le nombre de polygones par classe
polygon_counts = gdf['strate'].value_counts().reset_index()
polygon_counts.columns = ['Classe', 'Nombre de polygones']

# Convertir 'Classe' en chaîne de caractères pour éviter les conflits de type
polygon_counts['Classe'] = polygon_counts['Classe'].astype(str)

# Créer un DataFrame avec toutes les classes (1, 2, 3, 4), même si elles sont absentes
all_classes = pd.DataFrame({'Classe': ['1', '2', '3', '4']})

# Fusionner avec les comptes réels
polygon_counts = all_classes.merge(polygon_counts, on='Classe', how='left').fillna(0)

# Dictionnaire de correspondance classe/couleur (personnalisable)
color_map = {
    '1': '#F5DEB3',  # Sol nu (tan, couleur personnalisée)
    '2': '#98FB98',  # Herbe (palegreen, personnalisée)
    '3': '#32CD32',  # Landes (limegreen, personnalisée)
    '4': '#228B22'   # Arbre (darkgreen, personnalisée)
}

# Créer le diagramme avec Plotly
fig_polygons = px.bar(
    polygon_counts,
    x='Classe',
    y='Nombre de polygones',
    color='Classe',
    color_discrete_map=color_map,
    title='Nombre de polygones par classe',
    labels={'Classe': 'Classe', 'Nombre de polygones': 'Nombre de polygones'},
    category_orders={'Classe': ['1', '2', '3', '4']}
)

# Personnaliser l'affichage des classes sur l'axe x
fig_polygons.update_xaxes(
    type='category',  # Forcer l'affichage en catégories discrètes
    tickvals=['1', '2', '3', '4'],  # Valeurs exactes à afficher
    ticktext=['1', '2', '3', '4']   # Libellés à afficher
)

# Enregistrer la figure
output_polygon_path = '/home/onyxia/work/results/figure/diag_baton_nb_poly_by_class.html'
os.makedirs(os.path.dirname(output_polygon_path), exist_ok=True)
fig_polygons.write_html(output_polygon_path)

# Afficher la figure dans le notebook
fig_polygons.show()


In [ ]:
# Calculer l'aire de chaque polygone (en m², si le CRS est projeté)
gdf['Aire'] = gdf.geometry.area

# Calculer l'aire totale par classe
pixel_counts = gdf.groupby('strate')['Aire'].sum().reset_index()
pixel_counts.columns = ['Classe', 'Aire totale']

# Créer le diagramme avec Plotly
fig_pixels = px.bar(
    pixel_counts,
    x='Classe',
    y='Aire totale',
    color='Classe',
    color_discrete_map=color_map,
    title='Aire totale par classe (proxy pour le nombre de pixels)',
    labels={'Classe': 'Classe', 'Aire totale': 'Aire totale (m²)'},
    category_orders={'Classe': ['1', '2', '3', '4']}
)

# Enregistrer la figure
output_pixel_path = '/home/onyxia/work/results/figure/diag_baton_nb_pix_by_class.html'
fig_pixels.write_html(output_pixel_path)

# Afficher la figure dans le notebook
fig_pixels.show()
